In [1]:
import tensorflow as tf
import numpy as np 
from sklearn.datasets import fetch_california_housing
from IPython.display import clear_output, Image, display, HTML

###### Do not modify here ###### 
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = graph_def
    #strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
###### Do not modify  here ######

###### Implement Data Preprocess here ######
housing = fetch_california_housing()
print("Shape of dataset:", housing.data.shape)
print("Shape of label:", housing.target.shape)

# seperate train and test data
nine_tenths_offset = int(len(housing.data)*0.9)
train_X = housing.data[:nine_tenths_offset]
train_Y = housing.target[:nine_tenths_offset]
test_X = housing.data[nine_tenths_offset:]
test_Y = housing.target[nine_tenths_offset:]

dim_of_train_data = train_X.shape[0]
dim_of_train_feature = train_X.shape[1]
train_Y = train_Y.reshape(dim_of_train_data, 1)
dim_of_test_data = test_X.shape[0]
dim_of_test_feature = test_X.shape[1]
test_Y = test_Y.reshape(dim_of_test_data, 1)

# setting linear regression placeholder
x = tf.placeholder(tf.float32, [dim_of_train_data, dim_of_train_feature])
y = tf.placeholder(tf.float32, [dim_of_train_data, 1])

# setting linear regression variable
W = tf.Variable(np.random.rand(dim_of_train_feature, 1), dtype=tf.float32, name='weight')  # change to matrix
b = tf.Variable([-.3], dtype=tf.float32, name='bias')

# Linear regression : y = W * x + b
liner_regression = tf.add(tf.matmul(x, W), b)  # change to matrix
loss_rate = tf.reduce_mean(liner_regression - y)
optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss_rate)

###### Implement Data Preprocess here ######

init = tf.global_variables_initializer()


###### Start TF session ######
with tf.Session() as sess:
    sess.run(init)

    for i in range(1000):
        sess.run(optimizer, feed_dict={x: train_X, y: train_Y})
    
    # evaluate training accuracy
    curr_W, curr_b, curr_loss = sess.run([W, b, loss_rate], {x: train_X, y: train_Y})
    print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))
    # todo explain graph
    # todo data preprocessing to improve loss_rate
    show_graph(tf.get_default_graph().as_graph_def())
###### Start TF session ######

Shape of dataset: (20640, 8)
Shape of label: (20640,)
W: [[ -3.86836853e+01]
 [ -2.90389465e+02]
 [ -5.36176414e+01]
 [ -1.07797909e+01]
 [ -1.43310840e+04]
 [ -2.93961964e+01]
 [ -3.54065369e+02]
 [  1.19459094e+03]] b: [-10.30014133] loss: -2.07033e+07
